In [2]:
import os
import mne
import torch

import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
, random_split
import numpy as np
#from mamba_simple import Mamba
from mamba_ssm import Mamba
import sys
local_path = r'C:\Users\A ZHAI\EEG_Mamba\mamba-main\mamba_ssm\modules'
sys.path.insert(0, local_path)
from mamba4EEG import MixerModel4EEG

: 

In [4]:
import mne
import numpy as np
import os
import torch

# 定义fif文件所在的目录
fif_directory = r'D:\SEED_1020\Tryresult'

# 获取目录下所有的fif文件路径
fif_files = [os.path.join(fif_directory, file) for file in os.listdir(fif_directory) if file.endswith('_epo.fif')]

# 用于存储所有文件的数据和标签
all_eeg_data = []
all_labels = []

# 遍历每个fif文件并提取数据和标签
for file_path in fif_files:
    # 提取标签（文件名中的第一个下划线前的部分）
    file_name = os.path.basename(file_path)
    label = int(file_name.split('_')[-2])

    label += 1
    
    # 读取fif文件
    epochs = mne.read_epochs(file_path, preload=True)
    eeg_data = epochs.get_data()  # 提取数据 (形状为 (n_epochs, n_channels, n_times))
    
    # 将每个eeg信号都赋予相同的标签
    all_eeg_data.append(eeg_data)
    all_labels.extend([label] * len(eeg_data))  # 每个信号都分配相同的标签

# 将所有数据合并成一个大的数组
all_eeg_data = np.concatenate(all_eeg_data, axis=0)

# 将数据和标签转换为PyTorch张量
eeg_tensor = torch.tensor(all_eeg_data, dtype=torch.float32)  # (eeg信号总数, 通道数, 时间步数)
eeg_tensor = eeg_tensor.permute(0, 2, 1)  # 重新排列为 (eeg信号总数, 时间步数, 通道数)
label_tensor = torch.tensor(all_labels, dtype=torch.int64)  # 标签张量

# 输出形状确认
print("EEG数据形状:", eeg_tensor.shape)  # 形状示例: torch.Size([3510, 时间步数, 通道数])
print("标签形状:", label_tensor.shape)  # 形状示例: torch.Size([3510])

Reading D:\SEED_1020\Tryresult\10_djc_eeg10_-1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     995.00 ms
        0 CTF compensation matrices available
Not setting metadata
296 matching events found
No baseline correction applied
0 projection items activated
Reading D:\SEED_1020\Tryresult\11_djc_eeg11_1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     995.00 ms
        0 CTF compensation matrices available
Not setting metadata
293 matching events found
No baseline correction applied
0 projection items activated
Reading D:\SEED_1020\Tryresult\12_djc_eeg12_0_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     995.00 ms
        0 CTF compensation matrices available
Not setting metadata
291 matching events found
No baseline correction applied
0 projection items activated
Reading D:\SEED_1020\Tryresult\13_djc_eeg13_-1_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     995.00 ms
        0 CTF c

In [6]:
eeg_tensor.shape[2]

tensor([2, 2, 2,  ..., 2, 2, 2])

In [8]:


# 检查是否有可用的 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 假设已经有数据集 data_texts 和标签 data_labels
# 例如：data_texts 是预处理后的文本张量，data_labels 是对应的情感标签张量
# data_texts 的形状为 (num_samples, sequence_length, feature_dimension)

# 数据集分割
dataset = TensorDataset(eeg_tensor, label_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 初始化Mamba模型
d_model = eeg_tensor.shape[2]  # 输入特征维度
num_classes = 3
model = MixerModel4EEG(d_model=d_model, n_layer=10, d_intermediate=1, device=device)

In [10]:
import torch
import torch.nn as nn

class MambaClassifier(nn.Module):
    def __init__(self, base_model, input_dim, num_classes):
        super(MambaClassifier, self).__init__()
        self.base_model = base_model  # 使用已有的 MixerModel4EEG
        self.classifier = nn.Linear(input_dim, num_classes)  # 分类层

    def forward(self, x):
        x = self.base_model(x)  # 获取 MixerModel4EEG 的输出
        x = self.classifier(x[:, -1, :])  # 使用最后一个时间步的输出进行分类
        return x

# 初始化新的 Mamba 分类模型
mamba_classifier = MambaClassifier(
    base_model=model, input_dim=d_model, num_classes=num_classes
).to(device)

# 损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mamba_classifier.parameters(), lr=0.001)

# 训练模型
num_epochs = 20
for epoch in range(num_epochs):
    # 训练阶段
    mamba_classifier.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mamba_classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_loader)

    # 验证阶段
    mamba_classifier.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = mamba_classifier(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            # 计算准确率
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_val_loss = total_val_loss / len(val_loader)
    accuracy = 100 * correct / total

    # 输出本轮结果
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, "
          f"Accuracy: {accuracy:.2f}%")

# 测试阶段
def evaluate_model(test_loader, model, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

# 创建测试数据加载器并评估模型
test_loader = DataLoader(val_dataset, batch_size=16)
evaluate_model(test_loader, mamba_classifier, device)


KeyboardInterrupt: 

In [14]:
# 导入 Analyzer 类
from analysis import Analyzer
import numpy as np

# 收集预测结果和真实标签
y_true = []
y_pred = []

mamba_classifier.eval()  # 将模型设置为评估模式
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = mamba_classifier(inputs)
        _, predicted = torch.max(outputs.data, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# 将 y_true 和 y_pred 转换为 numpy 数组
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 创建 Analyzer 实例并设置可视化选项
analyzer = Analyzer(plot=True, print_conf_mat=True)

# 假设你的类别名称是 0, 1, 2, ..., num_classes - 1
class_names = [str(i) for i in range(num_classes)]

# 使用 Analyzer 类进行分析和可视化
results = analyzer.analyze_classification(y_pred, y_true, class_names=class_names)

# 输出模型分析结果
print("Total Accuracy:", results["total_accuracy"])
print("Average Precision:", results["prec_avg"])
print("Average Recall:", results["rec_avg"])

# 调用多个可视化函数
# 1. 展示累计准确率如何随着允许的预测数量（k）的增加而改善
analyzer.show_acc_top_k_improvement(y_pred=np.argsort(y_pred, axis=1)[:, ::-1], y_true=y_true, k=5)

# 2. 绘制混淆矩阵
ConfMat = results['ConfMatrix']
analyzer.plot_confusion_matrix(ConfMat, label_strings=class_names, title="Confusion Matrix")

# 3. 绘制精确率和召回率的分布直方图
analyzer.prec_rec_histogram(results['precision'], results['recall'])

# 4. 生成并打印分类报告
classification_report = analyzer.generate_classification_report()
print("\nClassification Report:\n", classification_report)


NameError: name 'test_loader' is not defined